In [170]:
import h5py    
import numpy as np    
import os
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
sb.set(rc = {'figure.figsize':(50,38)})
sb.set(font_scale=3)

In [108]:
files = [f for f in os.listdir('.') if '.h5' in f]

In [188]:
def group_check(snp_feature_list,snp_comb_name_list):
    snp_groups_bool = []
    for k in snp_comb_name_list:
        snp_groups_bool.append('_' not in k)
    snp_groups_bool = sum(([ss]*2 for ss in snp_groups_bool),[])
    group1 = snp_feature_list[snp_groups_bool]
    group1_overall = np.max(group1,axis=0)
    
    group2_idxs = [not idx for idx in snp_groups_bool]
    group2 = snp_feature_list[group2_idxs]
    group2_overalls = np.log(np.subtract(group2,group1_overall))
    
    return group1_overall,group2_overalls,group2_idxs


def correlation_check(snp_feature_list, snp_comb_name_list):
    
    filtered_combs = []
    for snp1 in range(0,len(snp_feature_list)):
        for snp2 in range(0,len(snp_feature_list)):
            corr,_ = pearsonr(snp_feature_list[snp1],snp_feature_list[snp2])
            if -0.5 <= corr <= 0.5:
                filtered_combs.append(snp1)
                filtered_combs.append(snp2)
    #filtered_combs = list(set(filtered_combs))            
    return filtered_combs

In [189]:
comb_dict = {}
comb_arrays = {}
for ij in range(0,len(files)):
    f = h5py.File(files[ij],'r+')  
    combs_keys = list(f.keys())
    snp_list = np.zeros((len(combs_keys)*2,2002))
    for i in range(0,2*len(combs_keys),2):
        snp_list[i,:] = f[combs_keys[int(i/2)]][()][0,:]
        snp_list[i+1,:] = f[combs_keys[int(i/2)]][()][0,:]
    t_name = files[ij].split('.diff.h5')[0]

    #Method 1
    filt_list = correlation_check(snp_list,combs_keys)    
    #Method 2
    _,met2,idd = group_check(snp_list,combs_keys)    
    
    print("The number of significant ld snps for %s are %d" %(t_name,len(filt_list)))
    comb_dict[t_name] = [len(filt_list),len(set(filt_list))]

    comb_arrays[t_name] = met2
    print("The number of combinations from method 2 for %s is %d" %(t_name,len(met2)))
    
    
    
    xlabs = list(f.keys())
    xlabs = [[x]*2 for x in xlabs]
    xlabs =  [item for sublist in xlabs for item in sublist]
    prob_matrix = pd.DataFrame(snp_list)
    
    iddx = [idd[i] for i in range(0,len(idd),2)]
    xlabels = [combs_keys[i] for i in range(0,len(iddx)) if iddx[i]]
    xlabels = sum(([ss]*2 for ss in xlabels),[])
    if len(xlabels) != 0:
        sb.heatmap(met2,yticklabels = xlabels).set(title = files[ij].split('.diff.h5')[0])
        plt.savefig('.'.join([files[ij].split('.diff.h5')[0],'png']))
        plt.clf()

<ipython-input-188-17ea67ca3f8e>:11: RuntimeWarning: divide by zero encountered in log
  group2_overalls = np.log(np.subtract(group2,group1_overall))
<ipython-input-188-17ea67ca3f8e>:11: RuntimeWarning: invalid value encountered in log
  group2_overalls = np.log(np.subtract(group2,group1_overall))


The number of significant ld snps for rs10119 are 0
The number of combinations from method 2 for rs10119 is 2
The number of significant ld snps for rs10414043 are 71680
The number of combinations from method 2 for rs10414043 is 240
The number of significant ld snps for rs111789331 are 96
The number of combinations from method 2 for rs111789331 is 8
The number of significant ld snps for rs11556505 are 13090816
The number of combinations from method 2 for rs11556505 is 4072
The number of significant ld snps for rs12721046 are 45056
The number of combinations from method 2 for rs12721046 is 240
The number of significant ld snps for rs12721051 are 210944
The number of combinations from method 2 for rs12721051 is 494
The number of significant ld snps for rs12972156 are 72960
The number of combinations from method 2 for rs12972156 is 240
The number of significant ld snps for rs12972970 are 259840
The number of combinations from method 2 for rs12972970 is 494
The number of significant ld snps

<Figure size 3600x2736 with 0 Axes>

In [192]:
ij = 18
f = h5py.File(files[ij],'r+')  
combs_keys = list(f.keys())
snp_list = np.zeros((len(combs_keys)*2,2002))
for i in range(0,2*len(combs_keys),2):
    snp_list[i,:] = f[combs_keys[int(i/2)]][()][0,:]
    snp_list[i+1,:] = f[combs_keys[int(i/2)]][()][0,:]
t_name = files[ij].split('.diff.h5')[0]

    #Method 1
#filt_list = correlation_check(snp_list,combs_keys)    
    #Method 2
_,met2,idd = group_check(snp_list,combs_keys)    
    
print("The number of significant ld snps for %s are %d" %(t_name,len(filt_list)))
comb_dict[t_name] = [len(filt_list),len(set(filt_list))]

comb_arrays[t_name] = met2
print("The number of combinations from method 2 for %s is %d" %(t_name,len(met2)))
    
    

The number of significant ld snps for rs429358 are 128
The number of combinations from method 2 for rs429358 is 2


<ipython-input-188-17ea67ca3f8e>:11: RuntimeWarning: divide by zero encountered in log
  group2_overalls = np.log(np.subtract(group2,group1_overall))


In [194]:
combs_keys

['rs429358', 'rs429358_rs7412', 'rs7412']

In [185]:
ij=24
f = h5py.File(files[ij],'r+')  
combs_keys = list(f.keys())
snp_list = np.zeros((len(combs_keys)*2,2002))
for i in range(0,2*len(combs_keys),2):
    snp_list[i,:] = f[combs_keys[int(i/2)]][()][0,:]
    snp_list[i+1,:] = f[combs_keys[int(i/2)]][()][0,:]
t_name = files[ij].split('.diff.h5')[0]

    #Method 1
filt_list = correlation_check(snp_list,combs_keys)    
    #Method 2
_,met2,idd = group_check(snp_list,combs_keys)    
    
print("The number of significant ld snps for %s are %d" %(t_name,len(filt_list)))
comb_dict[t_name] = [len(filt_list),len(set(filt_list))]

comb_arrays[t_name] = met2
print("The number of combinations from method 2 for %s is %d" %(t_name,len(met2)))
    
    
    
xlabs = list(f.keys())
xlabs = [[x]*2 for x in xlabs]
xlabs =  [item for sublist in xlabs for item in sublist]
prob_matrix = pd.DataFrame(snp_list)
    
iddx = [idd[i] for i in range(0,len(idd),2)]
xlabels = [combs_keys[i] for i in range(0,len(iddx)) if iddx[i]]
xlabels = sum(([ss]*2 for ss in xlabels),[])
if len(xlabels) != 0:
    sb.heatmap(met2,yticklabels = xlabels).set(title = files[ij].split('.diff.h5')[0])
    plt.savefig('.'.join([files[ij].split('.diff.h5')[0],'png']))
    plt.clf()

0
The number of significant ld snps for rs6857 are 0
The number of combinations from method 2 for rs6857 is 0


In [164]:
plt.clf()

<Figure size 3600x2736 with 0 Axes>